In [6]:
#import libraries
import boto3
import json
# import os, io
# import re
import time
# from io import StringIO
# from io import BytesIO
# from os import listdir
# from os.path import isfile, join

# import pytesseract

# import pdfminer

# from pdfminer.converter import TextConverter
# from pdfminer.layout import LAParams #, LTTextBox, LTTextLine
# from pdfminer.pdfdocument import PDFDocument
# from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# from pdfminer.pdfpage import PDFPage
# from pdfminer.pdfpage import PDFTextExtractionNotAllowed
# from pdfminer.pdfparser import PDFParser

# from PIL import Image

# import textract

In [2]:
# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_JordiPlanas_Made_in_game.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
s3BucketName = "wri-testing"
region = 'eu-central-1'

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [3]:
def startJob(s3BucketName, objectName):
    response = None
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.start_document_text_detection(DocumentLocation={
        'S3Object': {'Bucket': s3BucketName,
                     'Name': objectName
                    }
        })
    return response["JobId"]
def isJobComplete(jobId):
 # For production use cases, use SNS based notification 
 # Details at: https://docs.aws.amazon.com/textract/latest/dg/api-async.html
    time.sleep(5)
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))
    return status
def getJobResults(jobId):
    pages = []
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
        while(nextToken):
            response = client.get_document_text_detection(JobId=jobId, NextToken=nextToken)
            pages.append(response)
            print("Resultset page recieved: {}".format(len(pages)))
            nextToken = None
            if('NextToken' in response):
                nextToken = response['NextToken']
    return pages


In [4]:
def AWS_pdf_to_text(key, s3BucketName):
    # Document
    documentName = key
    jobId = startJob(s3BucketName, documentName)
    print("Started job with id: {}".format(jobId))
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
    #print(response)
    # Print detected text
    policy = ""
    for resultPage in response:
        for item in resultPage["Blocks"]:
            if item["BlockType"] == "LINE":
                policy = policy + item["Text"] + "\n"
                print ('\033[94m' + item["Text"] + '\033[0m')
    return policy

In [7]:
i = 0
lower_limit = 0
upper_limit = lower_limit + 2
for obj in s3.Bucket(s3BucketName).objects.all():#.filter(Prefix='El Salvador'):
    if "India_image.pdf" in obj.key: #lower_limit < i < upper_limit:
        print(i, "**", obj.key)
        policy = AWS_pdf_to_text(obj.key, s3BucketName)
    i += 1

8 ** India_image.pdf
Started job with id: 9707b05660e2e6ac417a7cb648dd350a6525ffc6b73f08a938bc94bb380ae029
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
10/60 794 936/39
Registered No. 768/97
2/2/21
4/97/4
THE ASSAM GAZETTE
12/2/2017
EXTRAORDINARY
one $624 5/41
PUBLISHED BY AUTHORITY
oi 207 Thergi 7, 2/30/14 24 0/65/44 2000, 2 PITG 1922 (MT)
No. 207 Dispur, Tuesday, 24th October, 2000, 22nd Kartika, 1922 (S.E.)
GOVERNMENT OF ASSAM
ORDERS BY THE GOVERNOR
LEGISLATIVE DEPARTMENT : LEGISLATIVE BRANCH
The 22nd September, 2000
NOTIFICATION
No. LGL. 25/2000/14.-- The following Act of the Assam Legislative
Assembly which received the assent of the Governor is hereby published
for general information.
830 THE ASSAM GAZETTE, EX

In [ ]:
policy

# Checking whether the files are scanned images or true pdfs

In [ ]:
from io import StringIO
import os
s = StringIO()
s.write("abc")
pos = s.tell()
s.seek(0, os.SEEK_END)
print(s.tell())
s.seek(pos)

In [ ]:
onlyfiles = [f for f in listdir('../input/') if isfile(join('../input/', f))]

files = [f for f in onlyfiles if "~" not in f and "git" not in f]
for file in files:
    if "doc" in file:
        print("\n\n### ", file, " ###\n\n")
        try:
            print(textract.process('../input/' + file))
        except:
            print("Problemo")
            pass
        print("\n\n**************************\n\n")

In [ ]:

def is_image(file):
    fp = open(file, 'rb')
    # Create a PDF parser object associated with the file object.
    parser = PDFParser(fp)
    # Create a PDF document
    document = PDFDocument(parser)
    if not document.is_extractable:
        raise PDFTextExtractionNotAllowed
    else:
        return parser


In [ ]:
print(is_image('../input/India1.pdf'))